In [1]:
%matplotlib inline
import sys
# This is the proposed version, to be evaluated.
sys.path.append('./bdit_traffic_prophet/')
# Dummy version of Traffic Prophet at commit e4e3cfa, but with code folder name changed.
# This is the baseline version.
sys.path.append('./btp_litreview/')
import importlib
import matplotlib.pyplot as plt
import numpy as np
import knowyourdata as kyd

import pandas as pd

from traffic_prophet.data import SAMPLE_ZIP
from traffic_prophet.countmatch import reader
from traffic_prophet.countmatch import permcount as pc
from tp_comp.countmatch import reader as reader_comp
from tp_comp.countmatch import growthfactor as gf_comp

defaultcolours = plt.rcParams['axes.prop_cycle'].by_key()['color']

In [2]:
# Read and process counts using baseline version.

rdr = reader_comp.Reader(SAMPLE_ZIP)
rdr.read()
gf_comp.get_growth_factors(rdr)

In [3]:
# Read and process using proposed version.

tcs = reader.read(SAMPLE_ZIP)
pc.get_ptcs_sttcs(tcs)

In [4]:
# Check STTC daily counts and other properties.  These should be same in both `tcs` and `rdr`
# for those locations that aren't also PTCs.
tcs.sttcs

{-241: <traffic_prophet.countmatch.base.Count at 0x7f17081480f0>,
 -252: <traffic_prophet.countmatch.base.Count at 0x7f16d4735cf8>,
 -410: <traffic_prophet.countmatch.base.Count at 0x7f16d461a198>,
 -427: <traffic_prophet.countmatch.base.Count at 0x7f16d47216a0>,
 -446378: <traffic_prophet.countmatch.base.Count at 0x7f16d47214e0>,
 -487: <traffic_prophet.countmatch.base.Count at 0x7f16d461a2b0>,
 -170: <traffic_prophet.countmatch.base.Count at 0x7f17081255c0>,
 -680: <traffic_prophet.countmatch.base.Count at 0x7f16d4735c50>,
 170: <traffic_prophet.countmatch.base.Count at 0x7f16d4721908>,
 104870: <traffic_prophet.countmatch.base.Count at 0x7f16d47218d0>,
 -1978: <traffic_prophet.countmatch.base.Count at 0x7f16d4721780>}

In [5]:
for key in tcs.sttcs.keys():
    assert tcs.sttcs[key].count_id == rdr.sttcs[key].count_id
    assert tcs.sttcs[key].centreline_id == rdr.sttcs[key].centreline_id
    assert tcs.sttcs[key].direction == rdr.sttcs[key].direction
    assert tcs.sttcs[key].is_permanent == rdr.sttcs[key].is_permanent
    assert tcs.sttcs[key].data.equals(rdr.sttcs[key].data)

In [6]:
# For PTCs, we have to be slightly more involved.
for key in tcs.ptcs.keys():
    assert tcs.ptcs[key].count_id == rdr.ptcs[key].count_id
    assert tcs.ptcs[key].centreline_id == rdr.ptcs[key].centreline_id
    assert tcs.ptcs[key].direction == rdr.ptcs[key].direction
    assert tcs.ptcs[key].is_permanent == rdr.ptcs[key].is_permanent
    
    permyr = tcs.ptcs[key].perm_years
    sttcyr = list(set(tcs.ptcs[key].data.index.levels[0]) -
                  set(tcs.ptcs[key].perm_years))
    assert tcs.ptcs[key].data.loc[permyr].equals(rdr.ptcs[key].data['Daily Count'])
    assert tcs.ptcs[key].data.loc[sttcyr].equals(rdr.sttcs[key].data)

    assert (tcs.ptcs[key].adts['MADT'].loc[permyr, ['MADT', 'Days in Month']]
            .equals(rdr.ptcs[key].data['MADT']))
    assert tcs.ptcs[key].adts['AADT'].equals(rdr.ptcs[key].data['AADT'])
    assert tcs.ptcs[key].ratios['DoM_ijd'].equals(rdr.ptcs[key].data['DoM Factor'])
    
    assert np.isclose(tcs.ptcs[key].growth_factor, rdr.ptcs[key].growth_factor)
    assert np.allclose(tcs.ptcs[key]._growth_fit['fit_results'].params,
                       rdr.ptcs[key]._fit.params)